In [2]:
import pandas as pd
import re
from keybert import KeyBERT

/Users/yumatakechi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# fill null values with empty string to avoid error
cases_df = pd.read_csv('cases.csv')
cases_df = cases_df.dropna(subset=['body']).fillna('')

FileNotFoundError: [Errno 2] No such file or directory: 'cases.csv'

In [36]:
print(len(cases_df))

2241


In [39]:
# remove URLs and special texts
def clean_text(text):
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s.,?]', '', text)
    text = text.strip().lower()

    return text

In [41]:
# join the the relevant columns for each row to create a document
documents = []
for index, row in cases_df.iterrows():
    joined_text = ' '.join([(row['title']), (row['description']), (row['body'])])
    doc = clean_text(joined_text)
    
    documents.append(doc)

In [1]:
print(documents[:10])

NameError: name 'documents' is not defined

In [56]:
kw_model = KeyBERT()

In [57]:
def extract_keywords(doc):
    return kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), use_maxsum=True, nr_candidates=20, stop_words='english', top_n=5)

In [64]:
# extract the top 5 relevant keywords (phrases with 1 to 3 words) from each document
keywords = [extract_keywords(doc) for doc in documents]
print(keywords[9])

[('alsoparticipatory budgetingreferences', 0.4435), ('budget issues', 0.4472), ('city daly', 0.4669), ('city participatory', 0.488), ('californian municipalities', 0.5307)]


In [65]:
len(keywords)

10

In [66]:
# separate the tuple into phrases and scores, and storing phrases as words by splitting 
keyword_list = []
score_list = []
for keyword in keywords:
    keyword_list.append(keyword[0])
    score_list.append(keyword[1])

In [67]:
cases_df['keywords'] = keyword_list
cases_df['scores'] = score_list

ValueError: Length of values (10) does not match length of index (2241)